# Model Creation and Training

In [9]:
import pandas as pd
import numpy as np
import logging
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

logging.basicConfig(filename = "logs.log", format = "%(asctime)s -- %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.INFO)

In [10]:
### Load_Data ###
training_data = pd.read_csv("training_data/01Jan2022.csv")
training_data["Datetime"] = training_data["Datetime"].astype("datetime64").astype(int)
training_data.set_index("Datetime", inplace=True)
training_data.head()
percent_positive = (training_data["Response"].sum() / len(training_data)) * 100
X = training_data.loc[:, ["Open", "VWAP"]]
y = training_data.loc[:, "Response"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

logging.info("Dataset Upload successfully")
logging.info(f"X training set shape:{X_train.shape}")
logging.info(f"y training set shape:{y_train.shape}")
logging.info(f"X test set shape:{X_test.shape}")
logging.info(f"y test set shape:{y_test.shape}")
logging.info(f"Percent Positive Response {round(percent_positive,2)}%")
print("Dataset Upload successfully")
print(f"X training set shape:{X_train.shape}")
print(f"y training set shape:{y_train.shape}")
print(f"X test set shape:{X_test.shape}")
print(f"y test set shape:{y_test.shape}")
print(f"Percent Positive Response {round(percent_positive,2)}%")


Dataset Upload successfully
X training set shape:(431216, 2)
y training set shape:(431216,)
X test set shape:(107804, 2)
y test set shape:(107804,)
Percent Positive Response 41.73%


In [11]:
X_train = X_train.reset_index().drop("Datetime", axis = 1)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.expand_dims(X_train, axis = -1)


In [12]:
### Global Variables ####
batch_size = 256
sequence_len = 1440 # Looking at 24 hours worth of data to determine
epochs = 1
attention_heads = 4
projection_dim = 1441
dropout = 0.1
num_transformer_blocks = 2
mlp_units = [2048, 1024]
tranformer_mlp_units = [projection_dim * 2, projection_dim]


In [13]:
### Convert time to a vector that can be encoded to the features ###
class Time2Vector(tf.keras.layers.Layer):
    def __init__(self, kernal: int = 64):
        super().__init__(trainable = True,  name = "Time2VecLayer")
        self.kernal = kernal - 1
        
    def build(self, input_shape):
        ### Time to Vector Piecewise function ###
        
        self.weights_linear = self.add_weight(shape = (1, input_shape[1], 1), initializer = "uniform", trainable = True, name = "weights_linear")
        
        self.bias_linear = self.add_weight(shape = (1, input_shape[1], 1), initializer = "uniform", trainable = True, name = "bias_linear")
        
        self.weights_periodic = self.add_weight(shape = (1, input_shape[1],self.kernal), initializer = "uniform", trainable = True, name = "weights_periodic")
        
        self.bias_periodic = self.add_weight(shape = (1, input_shape[1],self.kernal), initializer = "uniform", trainable = True, name = "bias_periodic")
    
    def call(self, x):
        x = tf.expand_dims(x, axis = -1)
        time_linear = (self.weights_linear * x) + self.bias_linear
        # time_linear = tf.expand_dims(time_linear, axis = -1) #Expand dimensions to concat later
        
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        # time_periodic = tf.expand_dims(time_periodic, axis = -1)
        
        final_product = tf.concat([time_linear, time_periodic], axis = -1)
        final_product = tf.expand_dims(final_product, axis = 0)
        return final_product
    
    
# test_vector = tf.random.uniform(shape = (1440, 2), dtype = tf.float32)
# test_vector
# exampleTV = Time2Vector(64)(test_vector)
# exampleTV
        

In [14]:
def mlp_block(x, units):
    for unit in units:
        x = layers.Dense(unit, activation = tf.nn.gelu)(x)
        x = layers.Dropout(0.1)(x)
    return x
def transformer_encoder(inputs, attention_heads, projection_dim, dropout):
    ### Layer Normalization / Multihead Attention Layers ###
    x = layers.LayerNormalization(epsilon = 1e-6)(inputs)
    x = layers.MultiHeadAttention(num_heads = attention_heads, key_dim = projection_dim, dropout = dropout)(x,x)
    skip1 = layers.Add()([x, inputs])
    
    ### Feed Forward ###
    x = layers.LayerNormalization(epsilon = 1e-6)(skip1)
    x = mlp_block(x, tranformer_mlp_units)
    skip2 = layers.Add()([x,skip1])
    
    return skip2


def build_model():
    time_embeddings = Time2Vector(sequence_len)
    input = tf.keras.Input(shape = X_train.shape)
    time_embedding_input = tf.reshape(tf.squeeze(input), (X_train.shape[:-1]))
    x = input
    x = time_embeddings(time_embedding_input)
    x = tf.keras.layers.Concatenate(axis = -1)([input, x])
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, attention_heads, projection_dim, dropout)
    x = tf.squeeze(x)
    x = tf.keras.layers.GlobalAveragePooling1D(data_format = "channels_first")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = mlp_block(x, mlp_units)
    output = tf.keras.layers.Dense(1, activation = "softmax")(x)
    
    model = tf.keras.Model(inputs = input, outputs = output)

    return model

In [15]:
def train_model(model):
    optimizer = tf.optimizers.Adam(learning_rate=1e-3, decay = 1e-4)
    checkpoint_path = "/models/"
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = "val_Accuracy", save_best_only = True, save_weights_only = True)
    model.compile(optimizer=optimizer, 
                  loss = tf.keras.losses.CategoricalCrossentropy(),
                  metrics = [tf.keras.metrics.CategoricalAccuracy(name = "Accuracy")])
    
    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = epochs,
        steps_per_epoch = X_train.shape[0]//batch_size,
        callbacks = [checkpoint_callback],
        
    )
    
    return history

In [17]:
X_train

<tf.Tensor: shape=(431216, 2, 1), dtype=float64, numpy=
array([[[ 2.81168579],
        [ 2.82390008]],

       [[ 2.82820697],
        [ 2.82830922]],

       [[ 2.83872044],
        [ 2.83535149]],

       ...,

       [[-0.8303174 ],
        [-0.83019217]],

       [[-0.8307079 ],
        [-0.83134267]],

       [[-0.83126361],
        [-0.83087326]]])>

In [16]:
model = build_model()
training = train_model(model)

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None